In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
first_url = 'https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&sort=user_rating,desc&ref_=adv_prv'

In [4]:
page = requests.get(first_url)
soup = BeautifulSoup(page.content, "html.parser")
scraped_movies = soup.find_all('div', class_='lister-item')
scraped_movies

[<div class="lister-item mode-advanced">
 <div class="lister-top-right">
 <div class="ribbonize" data-caller="filmosearch" data-tconst="tt0111161"></div>
 </div>
 <div class="lister-item-image float-left">
 <a href="/title/tt0111161/"> <img alt="The Shawshank Redemption" class="loadlate" data-tconst="tt0111161" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BNDE3ODcxYzMtY2YzZC00NmNlLWJiNDMtZDViZWM2MzIxZDYwXkEyXkFqcGdeQXVyNjAwNDUxODI@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png" width="67"/>
 </a> </div>
 <div class="lister-item-content">
 <h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">1.</span>
 <a href="/title/tt0111161/">The Shawshank Redemption</a>
 <span class="lister-item-year text-muted unbold">(1994)</span>
 </h3>
 <p class="text-muted">
 <span class="certificate">13+</span>
 <span class="ghost">|</span>
 <span class="runtime">142 min</span>
 <span class="ghost">|</span>
 <spa

In [5]:
def get_data(scraped_movies):
    ls = []

    for movie in scraped_movies:
        try:
            img_url = movie.find_all('div', class_='lister-item-image')[0].find_all('img', src=True)[0]['loadlate']
            title =  movie.find_all('div', class_='lister-item-image')[0].find_all('img', src=True)[0]['alt']
            year = movie.find_all('span', class_='lister-item-year')[0].get_text()
            try:
                certificate = movie.find_all('span', class_='certificate')[0].get_text()
            except:
                certificate = None
            duration = movie.find_all('span', class_='runtime')[0].get_text()
            genre = movie.find_all('span', class_='genre')[0].get_text()
            imdb = movie.find_all('div', class_='ratings-bar')[0].find_all('strong')[0].get_text()
            try:
                metascore = movie.find_all('span', class_='metascore')[0].get_text()
            except:
                metascore = None
            overview = movie.find_all('p', class_='text-muted')[1].get_text()
            director = movie.find_all('p', class_='')[0].find_all('a')[0].get_text()
            actors = []
            k = 0
            for actor in movie.find_all('p', class_='')[0].find_all('a'):
                if k != 0:
                    actors.append(actor.get_text())
                k += 1
            actors = ",".join(actors)

            tmp_ls = [img_url, title, year, certificate, duration, genre, imdb, metascore, overview, director, actors]
            ls.append(tmp_ls)
        except:
            print(title)

    return ls

In [6]:
i = 0
ls = []
while True:
    if i == 0:
        page = requests.get(first_url)
        soup = BeautifulSoup(page.content, "html.parser")
        scraped_movies = soup.find_all('div', class_='lister-item')
        ls = get_data(scraped_movies)
    else:
        url = f"https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&sort=user_rating,desc&start={i+1}&ref_=adv_nxt"
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        scraped_movies = soup.find_all('div', class_='lister-item')
        tmp_ls = get_data(scraped_movies)
        ls = ls + tmp_ls
    i += 50
    print(i)
    if i == 10000:
        print("finished")
        break

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
finished


In [7]:
df = pd.DataFrame(ls, columns=["img_url", "title", "year", "certificate", "duration", "genre", "imdb", "metascore", "overview", "director", "actors"])

In [8]:
df.isna().sum()

img_url           0
title             0
year              0
certificate    2979
duration          0
genre             0
imdb              0
metascore      2007
overview          0
director          0
actors            0
dtype: int64

In [9]:
df.head(2)

,img_url,title,year,certificate,duration,genre,imdb,metascore,overview,director,actors
0,https://m.media-amazon.com/images/M/MV5BNDE3OD...,The Shawshank Redemption,(1994),13+,142 min,\nDrama,9.3,82,"\nOver the course of several years, two convic...",Frank Darabont,"Tim Robbins,Morgan Freeman,Bob Gunton,William ..."
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,(1972),18+,175 min,"\nCrime, Drama",9.2,100,"\nDon Vito Corleone, head of a mafia family, d...",Francis Ford Coppola,"Marlon Brando,Al Pacino,James Caan,Diane Keaton"


In [10]:
df['year'] = df['year'].apply(lambda x : x[1:5])

In [11]:
df['certificate'].unique()

array(['13+', '18+', None, 'PG', '16+', '15+', '7+', '13A', 'G', 'R',
       'Not Rated', '15A', 'PG-13', '10+', '7A', 'M/PG', '6+', '10A',
       'DİKKAT', 'M', 'NC-17', 'Approved', 'Unrated', 'GP', 'YETİŞKİN',
       '6A', '7+13A', '14+'], dtype=object)

In [12]:
df['genre'] = df['genre'].apply(lambda x : x.split('\n')[1])

In [13]:
df['overview'] = df['overview'].apply(lambda x : x.split('\n')[1])

In [14]:
df['actors'] = df['actors'].apply(lambda x : "\t".join((x.split(','))))

In [15]:
df.to_csv('data.csv', index=False)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   img_url      9999 non-null   object
 1   title        9999 non-null   object
 2   year         9999 non-null   object
 3   certificate  7020 non-null   object
 4   duration     9999 non-null   object
 5   genre        9999 non-null   object
 6   imdb         9999 non-null   object
 7   metascore    7992 non-null   object
 8   overview     9999 non-null   object
 9   director     9999 non-null   object
 10  actors       9999 non-null   object
dtypes: object(11)
memory usage: 859.4+ KB


In [17]:
df.head(5)

,img_url,title,year,certificate,duration,genre,imdb,metascore,overview,director,actors
0,https://m.media-amazon.com/images/M/MV5BNDE3OD...,The Shawshank Redemption,1994,13+,142 min,Drama,9.3,82,"Over the course of several years, two convicts...",Frank Darabont,Tim Robbins\tMorgan Freeman\tBob Gunton\tWilli...
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,18+,175 min,"Crime, Drama",9.2,100,"Don Vito Corleone, head of a mafia family, dec...",Francis Ford Coppola,Marlon Brando\tAl Pacino\tJames Caan\tDiane Ke...
2,https://m.media-amazon.com/images/M/MV5BMGRjOG...,Hababam Sinifi,1975,18+,87 min,"Comedy, Drama",9.2,None,"Lazy, uneducated students share a very close b...",Ertem Egilmez,Kemal Sunal\tMünir Özkul\tHalit Akçatepe\tTari...
3,https://m.media-amazon.com/images/M/MV5BMjEwND...,DAMaN,2022,None,121 min,"Adventure, Drama",9.1,None,"The film is set in 2015. Sid, is a young docto...",Lenka Debiprasad,Vishal Mourya\tKaran Kandhapan\tBabushan Mohan...
4,https://m.media-amazon.com/images/M/MV5BNzQ1OD...,Spider-Man: Across the Spider-Verse,2023,PG,140 min,"Animation, Action, Adventure",9.0,86,"Miles Morales catapults across the Multiverse,...",Joaquim Dos Santos,Kemp Powers\tJustin K. Thompson\tShameik Moore...
